In [1]:
import os
import autokeras as ak
import pandas as pd 
from numpy import load
import tensorflow as tf

print(os.listdir("data/"))

['dogs_vs_cats_labels.npy', 'dogs_vs_cats_photos.npy', 'test1', 'train']


In [30]:
x_train = load('data/dogs_vs_cats_photos.npy')
y_train = load('data/dogs_vs_cats_labels.npy')

x_train.shape
y_train.shape

In [24]:
filenames = os.listdir("data/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append('dog')
    else:
        categories.append('cat')
        
data = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

data.tail()

,filename,category
24995,dog.9995.jpg,dog
24996,dog.9996.jpg,dog
24997,dog.9997.jpg,dog
24998,dog.9998.jpg,dog
24999,dog.9999.jpg,dog


In [33]:

# Init image classifier.
clf = ak.ImageClassifier(max_trials=3)
# Feed the image classifier with training data.
clf.fit(x_train, y_train)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Train for 625 steps, validate for 157 steps
Epoch 1/1000


382/625 [=================>............] - ETA: 1:11:07 - loss: 0.6845 - accuracy: 0.531 - ETA: 23:52 - loss: 0.5724 - accuracy: 0.7812  - ETA: 14:25 - loss: 0.4757 - accuracy: 0.862 - ETA: 10:22 - loss: 0.3813 - accuracy: 0.901 - ETA: 8:06 - loss: 0.3032 - accuracy: 0.923 - ETA: 6:06 - loss: 0.2286 - accuracy: 0.94 - ETA: 4:54 - loss: 0.1829 - accuracy: 0.95 - ETA: 4:21 - loss: 0.1614 - accuracy: 0.95 - ETA: 3:55 - loss: 0.1444 - accuracy: 0.96 - ETA: 3:34 - loss: 0.1307 - accuracy: 0.96 - ETA: 3:16 - loss: 0.1193 - accuracy: 0.97 - ETA: 3:01 - loss: 0.1098 - accuracy: 0.97 - ETA: 2:48 - loss: 0.1016 - accuracy: 0.97 - ETA: 2:37 - loss: 0.0946 - accuracy: 0.97 - ETA: 2:28 - loss: 0.0885 - accuracy: 0.97 - ETA: 2:20 - loss: 0.0832 - accuracy: 0.97 - ETA: 2:12 - loss: 0.0784 - accuracy: 0.98 - ETA: 2:05 - loss: 0.0742 - accuracy: 0.98 - ETA: 1:59 - loss: 0.0704 - accuracy: 0.98 - ETA: 1:54 - loss: 0.0669 - accuracy: 0.98 - ETA: 1:49 - loss: 0.0638 - accuracy: 0.98 - ETA: 1:45 - loss: 0.

625/625 [==============================] - ETA: 11s - loss: 0.0072 - accuracy: 0.998 - ETA: 11s - loss: 0.0071 - accuracy: 0.998 - ETA: 10s - loss: 0.0071 - accuracy: 0.998 - ETA: 10s - loss: 0.0070 - accuracy: 0.998 - ETA: 10s - loss: 0.1827 - accuracy: 0.994 - ETA: 10s - loss: 0.2727 - accuracy: 0.989 - ETA: 10s - loss: 0.2895 - accuracy: 0.984 - ETA: 10s - loss: 0.2926 - accuracy: 0.979 - ETA: 10s - loss: 0.2947 - accuracy: 0.975 - ETA: 10s - loss: 0.2966 - accuracy: 0.975 - ETA: 10s - loss: 0.2985 - accuracy: 0.975 - ETA: 9s - loss: 0.3004 - accuracy: 0.975 - ETA: 9s - loss: 0.3031 - accuracy: 0.97 - ETA: 9s - loss: 0.3056 - accuracy: 0.97 - ETA: 9s - loss: 0.3080 - accuracy: 0.97 - ETA: 9s - loss: 0.3094 - accuracy: 0.97 - ETA: 9s - loss: 0.3106 - accuracy: 0.97 - ETA: 9s - loss: 0.3114 - accuracy: 0.97 - ETA: 9s - loss: 0.3113 - accuracy: 0.97 - ETA: 8s - loss: 0.3105 - accuracy: 0.97 - ETA: 8s - loss: 0.3092 - accuracy: 0.97 - ETA: 8s - loss: 0.3078 - accuracy: 0.97 - ETA: 8s - 

404/625 [==================>...........] - ETA: 21:07 - loss: 45.6729 - accuracy: 0.0000e+ - ETA: 5:26 - loss: 38.6079 - accuracy: 0.0000e+00 - ETA: 3:11 - loss: 26.7596 - accuracy: 0.0000e+0 - ETA: 2:17 - loss: 20.1859 - accuracy: 0.0000e+0 - ETA: 1:48 - loss: 16.1730 - accuracy: 0.0000e+0 - ETA: 1:30 - loss: 13.4180 - accuracy: 0.0000e+0 - ETA: 1:17 - loss: 11.4404 - accuracy: 0.0000e+0 - ETA: 1:08 - loss: 9.9829 - accuracy: 0.0000e+0 - ETA: 1:04 - loss: 9.2114 - accuracy: 0.0000e+ - ETA: 1:00 - loss: 8.5575 - accuracy: 0.0000e+ - ETA: 57s - loss: 7.9967 - accuracy: 0.0000e+00 - ETA: 54s - loss: 7.5105 - accuracy: 0.0000e+0 - ETA: 51s - loss: 7.0849 - accuracy: 0.0000e+0 - ETA: 49s - loss: 6.7093 - accuracy: 0.0000e+0 - ETA: 46s - loss: 6.2217 - accuracy: 0.0000e+0 - ETA: 43s - loss: 5.8070 - accuracy: 0.0500    - ETA: 41s - loss: 5.4499 - accuracy: 0.116 - ETA: 39s - loss: 5.1392 - accuracy: 0.173 - ETA: 37s - loss: 4.8663 - accuracy: 0.224 - ETA: 36s - loss: 4.6247 - accuracy: 0.26

625/625 [==============================] - ETA: 7s - loss: 1.0703 - accuracy: 0.86 - ETA: 7s - loss: 1.0734 - accuracy: 0.86 - ETA: 6s - loss: 1.0756 - accuracy: 0.86 - ETA: 6s - loss: 1.0774 - accuracy: 0.85 - ETA: 6s - loss: 1.0787 - accuracy: 0.85 - ETA: 6s - loss: 1.0802 - accuracy: 0.84 - ETA: 6s - loss: 1.0810 - accuracy: 0.84 - ETA: 6s - loss: 1.0821 - accuracy: 0.83 - ETA: 6s - loss: 1.0827 - accuracy: 0.83 - ETA: 6s - loss: 1.0832 - accuracy: 0.83 - ETA: 6s - loss: 1.0834 - accuracy: 0.82 - ETA: 6s - loss: 1.0837 - accuracy: 0.82 - ETA: 6s - loss: 1.0837 - accuracy: 0.82 - ETA: 6s - loss: 1.0834 - accuracy: 0.81 - ETA: 6s - loss: 1.0830 - accuracy: 0.81 - ETA: 6s - loss: 1.0825 - accuracy: 0.81 - ETA: 6s - loss: 1.0819 - accuracy: 0.80 - ETA: 5s - loss: 1.0815 - accuracy: 0.80 - ETA: 5s - loss: 1.0803 - accuracy: 0.80 - ETA: 5s - loss: 1.0789 - accuracy: 0.79 - ETA: 5s - loss: 1.0778 - accuracy: 0.79 - ETA: 5s - loss: 1.0762 - accuracy: 0.79 - ETA: 5s - loss: 1.0750 - accuracy

...

625/625 [==============================] - ETA: 8s - loss: 0.6984 - accuracy: 0.92 - ETA: 8s - loss: 0.6947 - accuracy: 0.92 - ETA: 8s - loss: 0.6912 - accuracy: 0.92 - ETA: 8s - loss: 0.6876 - accuracy: 0.92 - ETA: 8s - loss: 0.7486 - accuracy: 0.92 - ETA: 7s - loss: 1.0463 - accuracy: 0.92 - ETA: 7s - loss: 1.2595 - accuracy: 0.91 - ETA: 7s - loss: 1.3475 - accuracy: 0.91 - ETA: 7s - loss: 1.3927 - accuracy: 0.90 - ETA: 7s - loss: 1.4134 - accuracy: 0.90 - ETA: 7s - loss: 1.4160 - accuracy: 0.90 - ETA: 7s - loss: 1.4138 - accuracy: 0.89 - ETA: 7s - loss: 1.4110 - accuracy: 0.89 - ETA: 7s - loss: 1.4080 - accuracy: 0.88 - ETA: 7s - loss: 1.4049 - accuracy: 0.88 - ETA: 7s - loss: 1.4019 - accuracy: 0.88 - ETA: 7s - loss: 1.3987 - accuracy: 0.87 - ETA: 7s - loss: 1.3954 - accuracy: 0.87 - ETA: 7s - loss: 1.3919 - accuracy: 0.87 - ETA: 6s - loss: 1.3882 - accuracy: 0.87 - ETA: 6s - loss: 1.3842 - accuracy: 0.87 - ETA: 6s - loss: 1.3801 - accuracy: 0.87 - ETA: 6s - loss: 1.3758 - accuracy

444/625 [====================>.........] - ETA: 21:47 - loss: 31.2586 - accuracy: 0.0000e+ - ETA: 5:35 - loss: 19.7470 - accuracy: 0.0000e+00 - ETA: 3:16 - loss: 15.2193 - accuracy: 0.0000e+0 - ETA: 2:21 - loss: 12.3797 - accuracy: 0.0000e+0 - ETA: 1:51 - loss: 10.3846 - accuracy: 0.0000e+0 - ETA: 1:32 - loss: 8.8895 - accuracy: 0.0020    - ETA: 1:19 - loss: 7.7041 - accuracy: 0.01 - ETA: 1:10 - loss: 6.7864 - accuracy: 0.03 - ETA: 1:03 - loss: 6.0604 - accuracy: 0.08 - ETA: 57s - loss: 5.4730 - accuracy: 0.1741 - ETA: 53s - loss: 4.9896 - accuracy: 0.252 - ETA: 50s - loss: 4.7144 - accuracy: 0.297 - ETA: 47s - loss: 4.3555 - accuracy: 0.355 - ETA: 44s - loss: 4.0493 - accuracy: 0.405 - ETA: 42s - loss: 3.7850 - accuracy: 0.447 - ETA: 39s - loss: 3.5538 - accuracy: 0.484 - ETA: 38s - loss: 3.3479 - accuracy: 0.516 - ETA: 36s - loss: 3.1634 - accuracy: 0.545 - ETA: 34s - loss: 2.9922 - accuracy: 0.570 - ETA: 33s - loss: 2.8378 - accuracy: 0.593 - ETA: 32s - loss: 2.6977 - accuracy: 0.61

625/625 [==============================] - ETA: 5s - loss: 0.6816 - accuracy: 0.89 - ETA: 5s - loss: 0.6785 - accuracy: 0.89 - ETA: 5s - loss: 0.6755 - accuracy: 0.89 - ETA: 5s - loss: 0.6725 - accuracy: 0.89 - ETA: 5s - loss: 0.6696 - accuracy: 0.89 - ETA: 5s - loss: 0.6666 - accuracy: 0.89 - ETA: 5s - loss: 0.6637 - accuracy: 0.89 - ETA: 5s - loss: 0.6608 - accuracy: 0.89 - ETA: 5s - loss: 0.6580 - accuracy: 0.89 - ETA: 5s - loss: 0.6551 - accuracy: 0.89 - ETA: 4s - loss: 0.6524 - accuracy: 0.89 - ETA: 4s - loss: 0.6496 - accuracy: 0.89 - ETA: 4s - loss: 0.6468 - accuracy: 0.89 - ETA: 4s - loss: 0.6441 - accuracy: 0.89 - ETA: 4s - loss: 0.6414 - accuracy: 0.89 - ETA: 4s - loss: 0.6387 - accuracy: 0.89 - ETA: 4s - loss: 0.6360 - accuracy: 0.89 - ETA: 4s - loss: 0.6334 - accuracy: 0.90 - ETA: 4s - loss: 0.6307 - accuracy: 0.90 - ETA: 4s - loss: 0.6281 - accuracy: 0.90 - ETA: 4s - loss: 0.6243 - accuracy: 0.90 - ETA: 4s - loss: 0.6217 - accuracy: 0.90 - ETA: 4s - loss: 0.6192 - accuracy

359/625 [================>.............] - ETA: 22:46 - loss: 79.4295 - accuracy: 0.0000e+ - ETA: 5:50 - loss: 59.4329 - accuracy: 0.0000e+00 - ETA: 3:25 - loss: 40.0304 - accuracy: 0.0000e+0 - ETA: 2:27 - loss: 28.3636 - accuracy: 0.1719    - ETA: 1:55 - loss: 21.8495 - accuracy: 0.363 - ETA: 1:36 - loss: 17.7705 - accuracy: 0.482 - ETA: 1:22 - loss: 14.9804 - accuracy: 0.564 - ETA: 1:13 - loss: 12.9535 - accuracy: 0.623 - ETA: 1:05 - loss: 11.4115 - accuracy: 0.668 - ETA: 59s - loss: 10.2023 - accuracy: 0.704 - ETA: 54s - loss: 9.2277 - accuracy: 0.7329 - ETA: 51s - loss: 8.4239 - accuracy: 0.756 - ETA: 49s - loss: 7.9612 - accuracy: 0.770 - ETA: 47s - loss: 7.5479 - accuracy: 0.782 - ETA: 45s - loss: 7.1755 - accuracy: 0.793 - ETA: 44s - loss: 6.8387 - accuracy: 0.802 - ETA: 42s - loss: 6.5319 - accuracy: 0.811 - ETA: 41s - loss: 6.2518 - accuracy: 0.820 - ETA: 40s - loss: 5.9956 - accuracy: 0.827 - ETA: 39s - loss: 5.7596 - accuracy: 0.834 - ETA: 38s - loss: 5.5407 - accuracy: 0.84

KeyboardInterrupt: 

In [ ]:
results = clf.predict(x_test)